In [87]:
import pandas as pd
import bz2

In [5]:
file_bookings = "Documents/bookings.csv.bz2"
file_searches = "Documents/searches.csv.bz2"

In [186]:
data_bookings = pd.read_csv(file_bookings, sep="^", compression="bz2", chunksize=1000000)
data_searches = pd.read_csv(file_searches, sep="^", compression="bz2", chunksize=1000000)

<ul>
<li>Distribute bookings and searches files over a million lines files</li>
</ul>

In [195]:
def create_chunk_files(data_reader, str_name):
    i = 0
    for chunk in data_reader:
        fle = open('Documents/{}_chunks_{}'.format(str_name, i), mode='w')
        chunk.to_csv(fle, sep='^', index=False)
        print(i),
        fle.close()
        i += 1
        
create_chunk_files(data_bookings, 'bookings')
create_chunk_files(data_searches, 'searches')

0 1 2 3 4 5 6 7 8 9 10 11 

C:\Users\Kenza\Anaconda2\lib\site-packages\ipykernel\ipkernel.py:175: DtypeWarning: Columns (40,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  shell.run_cell(code, store_history=store_history, silent=silent)
C:\Users\Kenza\Anaconda2\lib\site-packages\ipykernel\ipkernel.py:175: DtypeWarning: Columns (29,30,31,32,33,34,35,36,37,38,44) have mixed types. Specify dtype option on import or set low_memory=False.
  shell.run_cell(code, store_history=store_history, silent=silent)


12 13 14 15 16 17 18 19 20


<ul>
<li>Preprocess the bookings by extracting possible tupes (date, origin, destination).</li>
<li>Taking date into account as, e.g. a search made after a booking is not a search that ended up to a booking, so date needs to match as well.</li>
<li>Create tuples and save them in file.</li>
</ul>

In [162]:
import string
list_tuples = list()

def get_tuples(x):
    cre_date = string.split(x['cre_date'], sep=' ')[0]
    global list_tuples
    tuple_info = [cre_date, x['dep_port'], x['arr_port']]
    if tuple_info not in list_tuples:
        list_tuples.append(tuple_info)

In [163]:
import pickle
fle = open("Documents/tuples", "wb")
for booking_file_index in range(11): #we have 11 bookings chunks
    data_bookings = pd.read_csv('Documents/bookings_chunks_{}'.format(booking_file_index), sep='\s*\^\s*', engine='python')
    print('loaded file nb {}'.format(booking_file_index))
    i = 0
    while i < len(data_bookings):   #process each chunk in groups of 1000 lines
        list_tuples = list()
        data_chunk = data_bookings[i:(i+1000)]
        i += 1000
        if i % 100000 == 0:
            print(i),
        f = data_chunk.apply(get_tuples, axis=1)   #apply get_tuples() on the different rows to extract the tuples
        pickle.dump(list_tuples, fle)
fle.close()

loaded file nb 0
100000 200000 300000 400000 500000 600000 700000 800000 900000 1000000 loaded file nb 1
100000 200000 300000 400000 500000 600000 700000 800000 900000 1000000 loaded file nb 2
100000 200000 300000 400000 500000 600000 700000 800000 900000 1000000 loaded file nb 3
100000 200000 300000 400000 500000 600000 700000 800000 900000 1000000 loaded file nb 4
100000 200000 300000 400000 500000 600000 700000 800000 900000 1000000 loaded file nb 5
100000 200000 300000 400000 500000 600000 700000 800000 900000 1000000 loaded file nb 6
100000 200000 300000 400000 500000 600000 700000 800000 900000 1000000 loaded file nb 7
100000 200000 300000 400000 500000 600000 700000 800000 900000 1000000 loaded file nb 8
100000 200000 300000 400000 500000 600000 700000 800000 900000 1000000 loaded file nb 9
100000 200000 300000 400000 500000 600000 700000 800000 900000 1000000 loaded file nb 10


<ul>
<li>read tuples and look for the searches entries that match the booking tuples in date, origin and destination of flight</li>
</ul>

In [196]:
fle_date = open("Documents/tuples", 'rb')
tuples_date = list()
while 1:
    try:
        list_chunk = pickle.load(fle_date)
        tuples_date += list_chunk
    except EOFError:
        break
fle_date.close()

In [197]:
def find_matches_date(x):
    global tuples_date
    global res
    tuple_info = [x['Date'], x['Origin'], x['Destination']]
    #tuple_info = [x['Origin'], x['Destination']]
    if tuple_info in tuples_date:
        res.append(1)
    else:
        res.append(0)

In [205]:
file_search_new = open("Documents/searches_with_matches_date.csv", 'w')
for searching_file_index in range(21):
    data_searches = pd.read_csv('Documents/searches_chunks_{}'.format(searching_file_index), sep='^', engine='python')
    print('loaded file nb {}'.format(searching_file_index))
    
    i = 0
    res = list()
    while i < len(data_searches):
        data_chunk_search = data_searches[i:(i+100)]
        i += 100
        f = data_chunk_search.apply(find_matches_date, axis=1)
        
        if i % 100:
            print(i),
    data_searches.loc[:,'MatchesBookings'] = pd.Series(res)
    if searching_file_index == 0:
        data_searches.to_csv(file_search_new, sep='^', index=False)
    else:
        data_searches.to_csv(file_search_new, sep='^', index=False, header=False)
    break

file_search_new.close()

loaded file nb 0


KeyboardInterrupt: 